In [8]:
##YuYamaoka
##Rankエラー対策。真っ黒なマスクしかない画像を削除する。
##データセットにアクセスして、マスク画像からオブジェクトのない画像を見つけて削除する。

#path define
path_sum = ["mask"]
path_sum2 = ["mask","image"]

#Parameter Define
model_type = 'cyto'
chan = [0,0]#

import os
from tqdm import tqdm
import glob
import cv2
import numpy as np
import copy
from cellpose import models, io

#function define
def blob_detection(mask,class_id: int):
    """
        opencvのブロブ検出関数の戻り値が微妙に変わってる気がするのでダウングレード推奨
        最新はcv2.__version == 4.5.1なので pip install -U opencv-python==3.4.13

        N個の物体が入っている一枚の二値画像(height, width, 1)から,
        opencvのブロブ検出関数を利用して (height, widht, N)のNチャネル画像を生成する
        各チャネルが１つの物体のマスク情報となっている

        cls_idxsは各チャネルのマスク情報が示す物体のクラスidのリストとなっている
        ※もとのブログではすべての物体がcell(id=1)だったのでcls_idxsはnp.ones(N)でよかった
    """
    data = mask
    labels = []
    for label in np.unique(data):
        #: ラベルID==0は背景
        if label == 0:
            continue
        else:
            labels.append(label)

    if len(labels) == 0:
        #: 対象オブジェクトがない場合はNone
        #print(mask_path + "label is None!")
        return None, None
    else:
        mask = np.zeros((mask.shape)+(len(labels),), dtype=np.uint8)
        for n, label in enumerate(labels):
            mask[:, :, n] = np.uint8(data == label)
        cls_idxs = np.ones([mask.shape[-1]], dtype=np.int32) * class_id

        return mask, cls_idxs

def check_dataset(folder_path):
    files = glob.glob(folder_path + "/image/*.png")

    for img_path in tqdm(files):
        mask_exist = False
        for i in range(len(path_sum)):
            file_name = os.path.basename(img_path)
            mask_path = os.path.join(folder_path,path_sum[i],file_name)
            mask_id = i
            mask = img_to_cellpose(img_path)
            mask,class_id = blob_detection(mask,mask_id)
            if class_id is not None:
                mask_exist = True


        #全てのマスク画像にオブジェクトが確認されなかった場合、そのファイルを削除
        if(mask_exist == False):
            print(img_path)
            print("***********************")
            for j in range(len(path_sum2)):
                os.remove(os.path.join(folder_path,path_sum2[j],file_name))
                
def img_to_cellpose(img_path):
    """
    Input:
        img_path : (string) Image file PATH
    Return:
        saved inference data at file PATH.
    """
    model = models.Cellpose(gpu=False, model_type=model_type)
    img = io.imread(img_path)
    mask, flows, styles, diams = model.eval(img, diameter=None, channels=chan)

    # save results so you can load in gui
    #io.masks_flows_to_seg(img, masks, flows, diams, img_path, chan)

    #save results as png
    #plt.imsave("test.png",masks)

    return mask


In [ ]:


#main
check_dataset("./")



  5%|█▉                                        | 10/216 [00:40<13:25,  3.91s/it]/home/w-sugimr/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/w-sugimr/anaconda3/lib/python3.9/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
  5%|██▏                                       | 11/216 [00:43<12:12,  3.57s/it]

.//image/32CTX day14.png
***********************


  7%|███                                       | 16/216 [01:02<11:36,  3.48s/it]

.//image/0CTX day14.png
***********************


  8%|███▎                                      | 17/216 [01:07<13:06,  3.95s/it]